<a href="https://colab.research.google.com/github/allyhu6/language-acquisition/blob/main/MLLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
#to find he directory to unzip the data
import os
import pandas as pd
os.listdir()

cwd = os.getcwd()


import zipfile
with zipfile.ZipFile(cwd + "/Brown.zip", 'r') as zip_ref:
    zip_ref.extractall(cwd)

In [48]:
#iterating through the files and putting utterances into one file
import numpy as np
directory = "Brown"

for filename in os.listdir(directory):
  if filename in ['Adam', 'Eve', 'Sarah']:
    for file in os.listdir(directory + '/' + filename):
      #print(file)
      child_age = file[0:4]
      if file.endswith(".cha"):
          f = open(directory + '/' + filename +'/' + file)
          lines = f.read()
          fileOutput = open('alldata.csv','a')
          fileLines = lines.split('\n')
          for line in fileLines: # loop through each entry of that array (i.e., each line)
            if len(line)>0 and line[0:4] == '*CHI': # we only want the child tier
              line = line.replace('(', '')
              line = line.replace(')', '')
              lineContent = line.split('\t')[1] # second element of array, using tab character
              #lineWords = lineContent.split(' ') # get all words (split by space)
              fileOutput.write(str(child_age)+ ',' + str(lineContent) +'\n')
              
    else:
      fileOutput.close()

In [49]:
#fp = open("alldata.csv")
#content = fp.read()
#print(content)

In [69]:
#splitting the datafram and dropping lines where the words are indiscernable 
df = pd.read_csv("alldata.csv",names=["age", "utterance"])

#print(df.dtypes)
df[~df.utterance.str.contains("xxx")]
df[~df.utterance.str.contains("yyy")]
df['age'] = df['age'].astype(str)
df[df['age'].str.isnumeric()]
#print(df['age'].value_counts())

ages = []
for age in df['age']:
  if len(age) > 1:
    if int(age[:2]) >= 5:
      ages.append(int(age[:1]) + 1)
  else:
    ages.append(int(age[:1]))

df['age'] = ages
df = df.sample(frac = 1)

testing = df.sample(frac = .3)
training = df.sample(frac = .7)

testing.to_csv("test.csv")
training.to_csv("train.csv")

In [62]:
df['age'].value_counts()


3    201795
4    168801
5    109884
2     44379
6      7743
Name: age, dtype: int64

In [53]:
! pip install pytorch_transformers

from fastai.text import *
from fastai.metrics import *
import torch
import torch.nn as nn
from pytorch_transformers import RobertaTokenizer
from pytorch_transformers import RobertaModel

In [54]:

# To install the package "pytorch-transformers"
! pip install pytorch-transformers pendulum

import pendulum
from fastai.text import *
from fastai.metrics import *
import torch
import torch.nn as nn
from pytorch_transformers import RobertaTokenizer
from pytorch_transformers import RobertaModel

# Garbage Collector
import gc 

In [75]:
#config part
class Config(dict):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    for k, v in kwargs.items():
      setattr(self, k, v)

  def set(self, key, val):
    self[key] = val
    setattr(self, key, val)

config = Config(
  seed = 18,
  roberta_model_name = 'roberta-large',
  max_lr = 1e-5,
  epochs = 1,
  max_seq_len = 400,
  bs = 4,
  valid_pct = .2,
  hidden_dropout_prob = .05,
  hidden_size = 1024,
  num_labels = 5,
  start_tok = "<s>",
  end_tok = "</s>",
  model_path = 'Model_Roberta.pkl',
  pred_path = 'Prediction_Roberta.csv',
  train_file_path = 'train.csv',
  test_file_path = 'test.csv',
  text_column_name = 'utterance',
  target_column_name = 'age'
)

In [77]:

class FastAiRobertaTokenizer(BaseTokenizer):
  def __init__(self, tokenizer: RobertaTokenizer, max_seq_len: int=128, **kwargs):
    self._pretrained_tokenizer = Tokenizer()
    self.max_seq_len = max_seq_len
  def __call__(self, *args, **kwargs):
    return self
  def tokenizer(self, t:str) -> List[str]:
    return [config.start_tok] + self._pretrained_tokenizer.process_text(t, BaseTokenizer(lang="en"))[:self.max_seq_len - 2] + [config.end_tok]


roberta_tok = RobertaTokenizer.from_pretrained(config.roberta_model_name)
fastai_tokenizer = Tokenizer(tok_func = FastAiRobertaTokenizer(roberta_tok, max_seq_len = config.max_seq_len), pre_rules=[], post_rules=[])

path = Path()
roberta_tok.save_vocabulary(path)

with open('vocab.json', 'r') as f:
    roberta_vocab_dict = json.load(f)

fastai_roberta_vocab = Vocab(list(roberta_vocab_dict.keys()))

class RobertaTokenizerProcessor(TokenizeProcessor):
  def __init__(self, tokenizer):
    super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class RobertaNumericalProcessor(NumericalizeProcessor):
  def __init__(self, *args, **kwards):
    super().__init__(*args, vocab=fastai_roberta_vocab, **kwards)


def get_roberta_processor(tokenizer: Tokenizer=None, vocab:Vocab=None):
  return [RobertaTokenizerProcessor(tokenizer=tokenizer), NumericalizeProcessor(vocab=vocab)]



In [78]:
class RobertaDataBunch(TextDataBunch):
  @classmethod
  def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=64, val_bs:int=None, pad_idx=1,
    pad_first=True, decive:torch.device=None, no_check:bool=False, backwards:bool=False,
    l_tfms:Optional[Collection[Callable]]=None, **dl_kwargs) -> DataBunch:
    datasets = cls.__init__ds(train_ds, valid_ds, test_ds)
    val_bs = ifnone(val_bs, bs)
    collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
    train_sampler = SortishSample(dataset[0].x, key=lambda t:len(datasets[0][t][0].data), bs=bs)
    train_dl = Dataloader(datasets[0], batch_size=bs, sample= train_sampler, drop_last = True, **dl_kwargs)
    dataloaders = [train_dl]
    for ds in datasets[1:]:
      lengths = [len(t) for t in ds.x.items]
      sampler = SortSampler(ds.x, key = lengths.__getitem__)
      dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
    return cls(*dataloaders, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

class RobertaTextList(TextList):
  _bunch = RobertaDataBunch
  _label_cls = TextList


train_df = pd.read_csv(config.train_file_path)
test_df = pd.read_csv(config.test_file_path)
feat_cols = config.text_column_name
label_cols = config.target_column_name



processor = get_roberta_processor(
    tokenizer = fastai_tokenizer, vocab=fastai_roberta_vocab
)

data = TextList.from_df(train_df, ".", cols=feat_cols, processor=processor)\
  .split_by_rand_pct(valid_pct=config.valid_pct, seed=config.seed)\
  .label_from_df(cols=label_cols, label_cls=CategoryList)\
  .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor))\
  .databunch(bs=config.bs, pad_first=False, pad_idx=0)


del train_df
del test_df
gc.collect()



/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


0

In [79]:
# defining our model architecture
class CustomRobertaModel(nn.Module):
    def __init__(self, num_labels=5):
        super(CustomRobertaModel, self).__init__()
        self.num_labels = num_labels  # get number of labels
        self.roberta = RobertaModel.from_pretrained(
            config.roberta_model_name)  # get pre-trained model
        # set up percentage of drop
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # defining final output layer
        self.classifier = nn.Linear(config.hidden_size, num_labels)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.roberta(
            input_ids, token_type_ids, attention_mask)
        #print(pooled_output.size())
        logits = self.classifier(pooled_output)
        return logits


roberta_model = CustomRobertaModel()
learn = Learner(data, roberta_model, metrics=[accuracy]) #  use acc as evaluation metrics
learn = learn.to_fp16()  # train using half precision (instead of float 32) which can help to speedup

gc.collect() # clean the memory before modelling
learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(config.epochs, max_lr=config.max_lr) # train on one epoch
learn.export(config.model_path)  # store the model

epoch,train_loss,valid_loss,accuracy,time
0,1.317725,1.328352,0.377971,5:48:15


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [81]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
  learn.model.roberta.eval()
  preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
  sampler = [i for i in data.dl(ds_type).sampler]
  reverse_sampler = np.argsort(sampler)
  ordered_preds = preds[reverse_sampler, :]
  pred_values = np.argmax(ordered_preds, axis=1)
  return ordered_preds, pred_values

test_preds = get_preds_as_nparray(DatasetType.Test)

test_df = pd.read_csv(config.test_file_path)

prediction = test_preds[1] + 1
test_df[config.text_column_name] = prediction
test_df.drop([config.target_column_name], axis=1, inplace=True)
test_df.to_csv(config.pred_path, index=False)